In [3]:
# Re-run all model code after environment reset

import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Define sets
U = ['u1']
S = ['s1']
T = [1]
A = [0, 1, 2]  # 0 = hub, 1-2 = attractions
V = ['walk', 'bike']

# Parameters
Q = 1000  # minutes available per day
speed = {'walk': 0.05, 'bike': 0.15}  # km per minute
tv = {0: 0, 1: 60, 2: 90}  # visit time in minutes
#time_window = {0: (0, 1440), 1: (60, 300), 2: (120, 360)}
time_window = {0: (0, 1440), 1: (0, 1440), 2: (0, 1440)}
avail = {'u1': {'walk': 1, 'bike': 1}}
budget = {'u1': 400}  # budget in monetary units
BIG_M = 1e4

# Distances in km
d = np.array([
    [0.1, 2.0, 3.0],
    [2.0, 0.1, 1.5],
    [3.0, 1.5, 0.1]
])
# Travel cost (e.g., distance + mode-based flat rate)
c_cost = {v: d + (2 if v == 'bike' else 0) for v in V}

# Compute travel times for each vehicle
t_time = {}
for v in V:
    t_time[v] = d / speed[v]  # time = distance / speed

# Model
m = gp.Model("personalized_itinerary")

# Decision variables
x = m.addVars(U, S, T, A, A, V, vtype=GRB.BINARY, name="x")
y = m.addVars(U, S, T, A, vtype=GRB.BINARY, name="y")
arr = m.addVars(U, S, T, A, vtype=GRB.CONTINUOUS, name="arr")

# Objective: maximize total places visited (excluding hub)
m.setObjective(gp.quicksum(y[u, s, t, j] for u in U for s in S for t in T for j in A if j != 0), GRB.MAXIMIZE)
#??

# Constraints

for u in U:
    for s in S:
        for t in T:
            # Daily time constraint
            m.addConstr(
                gp.quicksum(t_time[v][i][j] * x[u, s, t, i, j, v] for i in A for j in A for v in V)
                + gp.quicksum(tv[j] * y[u, s, t, j] for j in A)
                <= Q, name=f"time_budget_{u}_{s}_{t}"
            )
            #
            #m.addConstr(gp.quicksum(x[u, s, t, i, j, v] for i in A for j in A for v in V) <=1, name=f"route_{u}_{s}_{t}")

            # Budget constraint
            m.addConstr(
                gp.quicksum(c_cost[v][i][j] * x[u, s, t, i, j, v]  for i in A for j in A for v in V)
                <= budget[u], name=f"cost_budget_{u}")
            
            #differentiate between shared rides and private rides, add condition of same i and j
            
            # Start and end at hub
            m.addConstr(gp.quicksum(x[u, s, t, 0, j, v] for j in A if j != 0 for v in V) == 1, name=f"start_{u}_{s}_{t}")
            m.addConstr(gp.quicksum(x[u, s, t, j, 0, v] for j in A if j != 0 for v in V) == 1, name=f"end_{u}_{s}_{t}")

            for j in A:
                if j != 0:
                    # Flow conservation
                    m.addConstr(
                        gp.quicksum(x[u, s, t, i, j, v] for i in A for v in V) == y[u, s, t, j],
                        name=f"flow_in_{u}_{s}_{t}_{j}"
                    )
                    m.addConstr(
                        gp.quicksum(x[u, s, t, j, k, v] for k in A for v in V) == y[u, s, t, j],
                        name=f"flow_out_{u}_{s}_{t}_{j}"
                    )

                # Time windows
                m.addConstr(arr[u, s, t, j] >= time_window[j][0] * y[u, s, t, j], name=f"tw_start_{u}_{s}_{t}_{j}")
                m.addConstr(arr[u, s, t, j] <= time_window[j][1] * y[u, s, t, j], name=f"tw_end_{u}_{s}_{t}_{j}")

            for i in A:
                for j in A:
                    if i == j:
                        for v in V:
                            m.addConstr(x[u, s, t, i, j, v] == 0, name=f"no_loop_{u}_{s}_{t}_{i}_{j}_{v}")
                    if i != j:
                        for v in V:
                            travel = t_time[v][i][j]
                            #m.addConstr(arr[u, s, t, j] >= arr[u, s, t, i] + tv[i] + travel - BIG_M * (1 - x[u, s, t, i, j, v]))
                            

            # Vehicle availability
            for v in V:
                if not avail[u][v]:
                    for i in A:
                        for j in A:
                            m.addConstr(x[u, s, t, i, j, v] == 0, name=f"vehicle_restrict_{u}_{v}_{i}_{j}")
                    
# New constraint: only one visit to each location (j ≠ 0) across all stations and days
for u in U:
    for j in A:
        if j != 0:
            m.addConstr(
                gp.quicksum(y[u, s, t, j] for s in S for t in T) <= 1,
                name=f"one_visit_per_location_{u}_{j}")

# Solve
m.optimize()

# Collect results
solution = {
    "visits": [(u, s, t, j) for u in U for s in S for t in T for j in A if y[u, s, t, j].X > 0.5],
    "routes": [(u, s, t, i, j, v) for u in U for s in S for t in T for i in A for j in A for v in V if x[u, s, t, i, j, v].X > 0.5]
}

# Format output
def get_route_details(routes, visits, speeds, d):
    route_info = []
    for (u, s, t, i, j, v) in routes:
        distance = d[i][j]
        time = distance / speeds[v]
        route_info.append({
            'user': u,
            'station': s,
            'day': t,
            'from_location': i,
            'to_location': j,
            'vehicle': v,
            'distance_km': round(distance, 2),
            'travel_time_min': round(time, 2),
        })

    visit_info = []
    for (u, s, t, j) in visits:
        visit_info.append({
            'user': u,
            'station': s,
            'day': t,
            'location': j,
            'visit_time_min': tv[j],
            'opening_window_min': time_window[j],
        })

    return route_info, visit_info

route_details, visit_details = get_route_details(
    solution["routes"], solution["visits"], speed, d
)

route_details, visit_details


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 22 rows, 24 columns and 91 nonzeros
Model fingerprint: 0x2538fb8e
Variable types: 3 continuous, 21 integer (21 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


([{'user': 'u1',
   'station': 's1',
   'day': 1,
   'from_location': 0,
   'to_location': 2,
   'vehicle': 'walk',
   'distance_km': np.float64(3.0),
   'travel_time_min': np.float64(60.0)},
  {'user': 'u1',
   'station': 's1',
   'day': 1,
   'from_location': 1,
   'to_location': 0,
   'vehicle': 'walk',
   'distance_km': np.float64(2.0),
   'travel_time_min': np.float64(40.0)},
  {'user': 'u1',
   'station': 's1',
   'day': 1,
   'from_location': 2,
   'to_location': 1,
   'vehicle': 'walk',
   'distance_km': np.float64(1.5),
   'travel_time_min': np.float64(30.0)}],
 [{'user': 'u1',
   'station': 's1',
   'day': 1,
   'location': 0,
   'visit_time_min': 0,
   'opening_window_min': (0, 1440)},
  {'user': 'u1',
   'station': 's1',
   'day': 1,
   'location': 1,
   'visit_time_min': 60,
   'opening_window_min': (0, 1440)},
  {'user': 'u1',
   'station': 's1',
   'day': 1,
   'location': 2,
   'visit_time_min': 90,
   'opening_window_min': (0, 1440)}])